In [1]:
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.chat.util import Chat, reflections

In [ ]:

preguntas_prueba = {
    'Quiero reservar una cita para el doctor Martínez, por favor.': 'consulta_turno',
    'Necesito agendar una hora médica para revisión de resultados.': 'consulta_turno',
    '¿Cuáles son los especialistas disponibles para una consulta?': 'informacion_medico',
    'Quisiera saber la información del médico que atiende en la clínica este mes.': 'informacion_medico',
    'Necesito cambiar mi cita para la próxima semana.': 'cancelar_turno',
    'Deseo anular la cita que tenía programada para mañana por la tarde.': 'cancelar_turno',
    '¿Cuáles son los horarios de atención para hoy?': 'pregunta_horario',
    '¿Está la clínica abierta los fines de semana?': 'pregunta_horario',
    'Por favor, confírmame la cita que tengo para el viernes a las 10 a.m.': 'confirmacion_turno',
    'Quiero verificar si mi cita médica está confirmada para el próximo martes.': 'confirmacion_turno',
    '¿Cuál es el área médica de la clínica?': 'pregunta_especialidad',
    '¿Qué tipo de especialidades clínicas ofrecen?': 'pregunta_especialidad',
    'Necesito asistencia, ¿podrían ayudarme?': 'ayuda_general',
    'Por favor, ¿me pueden brindar apoyo con una duda que tengo?': 'ayuda_general',
    '¿Cuál es la dirección de la clínica?': 'informacion_clinica',
    '¿Dónde están ubicados exactamente?': 'informacion_clinica',
    'Quiero poner una queja sobre la atención recibida la última vez.': 'reclamacion',
    'Tengo un problema con el servicio que me brindaron, ¿cómo puedo reportarlo?': 'reclamacion',
    '¿Cuáles son las formas de pago aceptadas para la consulta?': 'pregunta_pago',
    '¿Podría informarme sobre las tarifas de las consultas médicas?': 'pregunta_pago',
    'Gracias por la atención, me siento muy agradecido.': 'agradecimiento',
    'Mil gracias por la ayuda, estoy muy agradecido con ustedes.': 'agradecimiento',
    '¿Dónde puedo obtener los resultados de los análisis que me realicé?': 'consulta_resultados',
    'Quiero información sobre los informes médicos de las pruebas que me hicieron.': 'consulta_resultados',
    'Necesito renovar mi receta médica, ¿cómo debo proceder?': 'pregunta_receta',
    '¿Cómo puedo obtener una nueva receta médica para mis medicamentos?': 'pregunta_receta',
    '¿Cuánto tiempo de espera hay para una consulta?': 'pregunta_espera',
    '¿Cuál es la demora promedio para obtener un turno con el especialista?': 'pregunta_espera',
    '¿Qué coberturas médicas aceptan en la clínica?': 'consulta_seguro',
    '¿Cuáles son las aseguradoras con las que trabajan?': 'consulta_seguro',
    '¿Pueden proporcionarme un mapa o indicaciones para llegar a la clínica?': 'pregunta_ubicacion',
    '¿Cómo puedo llegar a la clínica desde el centro de la ciudad?': 'pregunta_ubicacion',
    'Necesito un certificado médico para justificar mi ausencia laboral.': 'pregunta_certificado',
    '¿Dónde puedo obtener un informe de ausencia por motivos médicos?': 'pregunta_certificado',
    '¿Cuál es el diagnóstico médico de mi enfermedad?': 'pregunta_diagnostico',
    '¿Tienen información sobre los resultados del diagnóstico que me realizaron?': 'pregunta_diagnostico',
    'Tengo una emergencia médica, ¿qué debo hacer?': 'pregunta_emergencia',
    '¿Cuál es el procedimiento en caso de una situación de urgencia médica?': 'pregunta_emergencia'
}
# Crear listas para almacenar preguntas y etiquetas
preguntas = []
etiquetas = []

# Iterar sobre el diccionario para obtener las preguntas y etiquetas
for pregunta, etiqueta in preguntas_prueba.items():
    preguntas.append(pregunta)
    etiquetas.append(etiqueta)


indice_etiquetas = {}

cont = 0

for x in etiquetas:
    if x not in indice_etiquetas:
        indice_etiquetas[x] = cont
        cont+=1

etiquetas_usar = []

for x in etiquetas:
    etiquetas_usar.append(indice_etiquetas[x])
    
etiquetas_usar

In [11]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch

# Cargar el modelo pre-entrenado y el tokenizador de BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=19)  # Reemplaza NUM_LABELS por el número de clases de tu tarea
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ejemplo de datos de entrenamiento (reemplaza con tus propios datos)
train_texts = preguntas
train_labels = etiquetas_usar


# Tokenizar los textos y convertirlos a IDs de token
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Convertir etiquetas a tensores de PyTorch
train_labels = torch.tensor(train_labels)

# Clase para el entrenamiento personalizado
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Crear instancias de datasets de entrenamiento y validación
train_dataset = CustomDataset(train_encodings, train_labels)

# Argumentos y configuraciones de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',  # Directorio de salida
    num_train_epochs=3,  # Número de epochs (ajusta según tus necesidades)
    per_device_train_batch_size=8,  # Tamaño de lote por dispositivo (ajusta según tus recursos)
    logging_dir='./logs',  # Directorio de logs
)

# Entrenamiento del modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()  # Iniciar el entrenamiento


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/15 [00:18<?, ?it/s]
                                               
100%|██████████| 15/15 [00:14<00:00,  1.04it/s]

{'train_runtime': 14.398, 'train_samples_per_second': 7.918, 'train_steps_per_second': 1.042, 'train_loss': 2.9408236185709637, 'epoch': 3.0}


TrainOutput(global_step=15, training_loss=2.9408236185709637, metrics={'train_runtime': 14.398, 'train_samples_per_second': 7.918, 'train_steps_per_second': 1.042, 'train_loss': 2.9408236185709637, 'epoch': 3.0})

In [12]:
# Suponiendo que tienes un texto para realizar predicciones
text_to_predict = "Otro turno"

# Tokenizar el texto para el modelo
inputs = tokenizer(text_to_predict, return_tensors="pt")

# Realizar la predicción utilizando el modelo entrenado
outputs = model(**inputs)

# Obtener la predicción de la etiqueta
predicted_label = torch.argmax(outputs.logits, dim=1).item()

# Mostrar la etiqueta predicha
print(f"Etiqueta predicha: {predicted_label}")


Etiqueta predicha: 14
